<a href="https://colab.research.google.com/github/sudheer-muthyala/Transformers_HuggingFace_Tensorflow/blob/main/transformers_lyric_generation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install datasets

In [2]:
import tensorflow as tf
from datasets import load_dataset
from transformers import GPT2TokenizerFast, DataCollatorForLanguageModeling, TFGPT2LMHeadModel, create_optimizer

In [ ]:
!pip install -q kaggle
!mkdir ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 /root/.kaggle/kaggle.json
!kaggle datasets download -d juicobowley/drake-lyrics
!unzip "/content/drake-lyrics.zip" -d "/content/dataset/"

In [4]:
filepath="/content/dataset/drake_data.csv"
dataset = load_dataset('csv', data_files=filepath)

In [5]:
dataset["train"][0]

{'album': 'Certified Lover Boy',
 'lyrics_title': 'Certified Lover Boy* Lyrics',
 'lyrics_url': 'https://genius.com/Drake-certified-lover-boy-lyrics',
 'lyrics': "[Verse]\nPut my feelings on ice\nAlways been a gem\nCertified lover boy, somehow still heartless\nHeart is only gettin' colder",
 'track_views': '8.7K'}

In [ ]:
model_id = "gpt2-medium"
tokenizer = GPT2TokenizerFast.from_pretrained(model_id)

In [7]:
len(dataset["train"]["lyrics"])

290

In [8]:
input = tokenizer(dataset["train"][111]["lyrics"], max_length=256, truncation=True, return_overflowing_tokens=True, return_length=True)
input

{'input_ids': [[58, 5317, 305, 60, 198, 1639, 973, 284, 869, 502, 319, 616, 198, 1639, 973, 284, 11, 345, 973, 284, 198, 10995, 198, 198, 58, 1925, 15125, 60, 198, 1639, 973, 284, 869, 502, 319, 616, 2685, 3072, 198, 26302, 12, 3847, 618, 345, 761, 616, 1842, 198, 14134, 502, 319, 616, 2685, 3072, 198, 26302, 12, 3847, 618, 345, 761, 616, 1842, 198, 1870, 314, 760, 618, 326, 46989, 698, 278, 198, 2504, 460, 691, 1612, 530, 1517, 198, 40, 760, 618, 326, 46989, 698, 278, 198, 2504, 460, 691, 1612, 530, 1517, 198, 198, 58, 13414, 325, 352, 60, 198, 23921, 1201, 314, 1364, 262, 1748, 11, 345, 198, 30074, 257, 8507, 329, 3511, 783, 198, 28172, 4206, 290, 314, 1254, 1364, 503, 198, 24151, 11, 345, 1392, 502, 866, 11, 345, 1392, 502, 15033, 503, 198, 6, 42323, 1683, 1201, 314, 1364, 262, 1748, 11, 345, 198, 10434, 276, 5762, 1342, 290, 467, 259, 6, 503, 517, 198, 9861, 13978, 286, 37392, 503, 319, 262, 9280, 4314, 198, 39, 648, 259, 6, 351, 617, 4813, 314, 1053, 1239, 1775, 878, 198, 198, 58,

In [9]:
MAX_LENGTH = 256
BATCH_SIZE = 6

In [10]:
to_waste = 0
for i in range(len(dataset["train"]["lyrics"])):
  try:
    input = tokenizer(dataset["train"][i]["lyrics"], max_length=MAX_LENGTH, truncation=True, return_overflowing_tokens=True, return_length=True)
    print(input["length"])
    for j in input["length"]:
      if j != MAX_LENGTH:
        to_waste += j
  except:
    print("----------------->", i)

print("to_waste--->", to_waste)

[33]
[252]
[256, 256, 256, 72]
[256, 256, 48]
[256, 256, 137]
[256, 256, 256, 159]
[256, 256, 256, 12]
[146]
[256, 256, 252]
[256, 256, 210]
[256, 256, 256, 122]
[256, 256, 256, 86]
[256, 256, 173]
[256, 256, 256, 249]
[256, 256, 256, 168]
[256, 256, 229]
[256, 256, 185]
[256, 256, 256, 256, 76]
[256, 256, 256, 138]
[256, 256, 256, 140]
[256, 256, 256, 155]
[256, 256, 256, 256, 236]
[256, 256, 256, 59]
[256, 256, 256, 167]
[256, 256, 115]
[256, 256, 256, 30]
[256, 256, 256, 79]
[256, 256, 256, 256, 71]
[256, 256, 256, 204]
[256, 256, 256, 226]
[256, 256, 256, 63]
[256, 256, 57]
[256, 229]
[256, 256, 256, 256, 168]
[256, 256, 256, 256, 35]
[256, 256, 256, 256, 82]
[256, 256, 242]
[256, 256, 107]
[256, 256, 185]
[256, 164]
[256, 256, 256, 75]
[256, 256, 256, 141]
[256, 253]
[256, 256, 256, 256, 152]
[256, 256, 27]
[256, 256, 256, 99]
[256, 256, 102]
[256, 256, 175]
[256, 256, 256, 55]
[256, 256, 256, 109]
[256, 256, 100]
[256, 256, 256, 79]
[256, 256, 256, 138]
[256, 256, 256, 61]
[256, 

In [11]:
def data_preprocess(example):
  try:
    outputs = tokenizer(example["lyrics"],
                        max_length=MAX_LENGTH,
                        truncation=True,
                        return_overflowing_tokens=True,
                        return_length=True)

    input = []
    for length, input_ids in zip(outputs["length"], outputs["input_ids"]):
      if length == MAX_LENGTH:
        input.append(input_ids)
        input_ids_bk = input_ids
    if len(input) != 0:
      for i in range(BATCH_SIZE - len(input)):
        input.append(input_ids_bk)

  except:
    input = []

  return {"input_ids": input}

In [12]:
tokenized_dataset = dataset.map(data_preprocess, remove_columns=dataset["train"].column_names)

In [13]:
tokenized_dataset

DatasetDict({
    train: Dataset({
        features: ['input_ids'],
        num_rows: 290
    })
})

In [ ]:
for i in range(290):
  print(len(tokenized_dataset["train"][i]["input_ids"]))

In [15]:
def filter_input_ids_len(example):
  if len(example["input_ids"]) == 6:
    return example

tokenized_full_dataset = tokenized_dataset.filter(filter_input_ids_len)

In [16]:
tokenized_full_dataset

DatasetDict({
    train: Dataset({
        features: ['input_ids'],
        num_rows: 270
    })
})

In [ ]:
tokenized_full_dataset["train"][0]

In [18]:
tokenizer.pad_token = tokenizer.eos_token
data_collator = DataCollatorForLanguageModeling(tokenizer, mlm=False, return_tensors="tf")

In [19]:
tf_dataset = tokenized_full_dataset["train"].to_tf_dataset(columns=["input_ids","attention_mask", "labels"],
                                                  collate_fn=data_collator,
                                                  shuffle=True,
                                                  batch_size=1)

In [20]:
for element in tf_dataset.take(1):
  print(element)

{'input_ids': <tf.Tensor: shape=(1, 6, 256), dtype=int64, numpy=
array([[[   58, 13414,   325, ...,   760,   345,   821],
        [  407,  6164,  8720, ...,   290,  1627,   705],
        [  368,   510,    11, ...,   284,    11,  1521],
        [  368,   510,    11, ...,   284,    11,  1521],
        [  368,   510,    11, ...,   284,    11,  1521],
        [  368,   510,    11, ...,   284,    11,  1521]]])>, 'attention_mask': <tf.Tensor: shape=(1, 6), dtype=int64, numpy=array([[1, 1, 1, 1, 1, 1]])>, 'labels': <tf.Tensor: shape=(1, 6, 256), dtype=int64, numpy=
array([[[   58, 13414,   325, ...,   760,   345,   821],
        [  407,  6164,  8720, ...,   290,  1627,   705],
        [  368,   510,    11, ...,   284,    11,  1521],
        [  368,   510,    11, ...,   284,    11,  1521],
        [  368,   510,    11, ...,   284,    11,  1521],
        [  368,   510,    11, ...,   284,    11,  1521]]])>}


In [21]:
def fix_attention_mask(input):
  return {"input_ids": input["input_ids"],
          "attention_mask": tf.ones([1, BATCH_SIZE, MAX_LENGTH]),
          "labels": input["labels"]}

train_dataset = tf_dataset.map(fix_attention_mask)

In [22]:
for element in train_dataset.take(1):
  print(element)

{'input_ids': <tf.Tensor: shape=(1, 6, 256), dtype=int64, numpy=
array([[[  58, 5317,  305, ...,  484, 1204,  329],
        [  58, 5317,  305, ...,  484, 1204,  329],
        [  58, 5317,  305, ...,  484, 1204,  329],
        [  58, 5317,  305, ...,  484, 1204,  329],
        [  58, 5317,  305, ...,  484, 1204,  329],
        [  58, 5317,  305, ...,  484, 1204,  329]]])>, 'attention_mask': <tf.Tensor: shape=(1, 6, 256), dtype=float32, numpy=
array([[[1., 1., 1., ..., 1., 1., 1.],
        [1., 1., 1., ..., 1., 1., 1.],
        [1., 1., 1., ..., 1., 1., 1.],
        [1., 1., 1., ..., 1., 1., 1.],
        [1., 1., 1., ..., 1., 1., 1.],
        [1., 1., 1., ..., 1., 1., 1.]]], dtype=float32)>, 'labels': <tf.Tensor: shape=(1, 6, 256), dtype=int64, numpy=
array([[[  58, 5317,  305, ...,  484, 1204,  329],
        [  58, 5317,  305, ...,  484, 1204,  329],
        [  58, 5317,  305, ...,  484, 1204,  329],
        [  58, 5317,  305, ...,  484, 1204,  329],
        [  58, 5317,  305, ...,  484

In [23]:
unbatched_dataset=train_dataset.unbatch()

for element in unbatched_dataset.take(1):
  print(element)

{'input_ids': <tf.Tensor: shape=(6, 256), dtype=int64, numpy=
array([[   58,  1925, 15125, ...,    11,   582,   338],
       [  523,   555, 34715, ...,   338,   257,  5802],
       [ 2551,   357,    44, ..., 15125,    25, 19460],
       [ 2551,   357,    44, ..., 15125,    25, 19460],
       [ 2551,   357,    44, ..., 15125,    25, 19460],
       [ 2551,   357,    44, ..., 15125,    25, 19460]])>, 'attention_mask': <tf.Tensor: shape=(6, 256), dtype=float32, numpy=
array([[1., 1., 1., ..., 1., 1., 1.],
       [1., 1., 1., ..., 1., 1., 1.],
       [1., 1., 1., ..., 1., 1., 1.],
       [1., 1., 1., ..., 1., 1., 1.],
       [1., 1., 1., ..., 1., 1., 1.],
       [1., 1., 1., ..., 1., 1., 1.]], dtype=float32)>, 'labels': <tf.Tensor: shape=(6, 256), dtype=int64, numpy=
array([[   58,  1925, 15125, ...,    11,   582,   338],
       [  523,   555, 34715, ...,   338,   257,  5802],
       [ 2551,   357,    44, ..., 15125,    25, 19460],
       [ 2551,   357,    44, ..., 15125,    25, 19460],
   

In [24]:
model = TFGPT2LMHeadModel.from_pretrained(model_id)
model.summary()

All PyTorch model weights were used when initializing TFGPT2LMHeadModel.

All the weights of TFGPT2LMHeadModel were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFGPT2LMHeadModel for predictions without further training.


Model: "tfgpt2lm_head_model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 transformer (TFGPT2MainLay  multiple                  354823168 
 er)                                                             
                                                                 
Total params: 354823168 (1.32 GB)
Trainable params: 354823168 (1.32 GB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [25]:
num_training_steps = len(unbatched_dataset)
optimizer, _ = create_optimizer(init_lr=5e-5, num_warmup_steps=1000, num_train_steps=num_training_steps,)
model.compile(optimizer=optimizer)

In [26]:
model.fit(unbatched_dataset, epochs=5)

Epoch 1/5


Cause: for/else statement not yet supported
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


Cause: for/else statement not yet supported
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
270/270 [==============================] - 464s 1s/step - loss: 3.2644
Epoch 2/5
270/270 [==============================] - 335s 1s/step - loss: 2.9748
Epoch 3/5
270/270 [==============================] - 335s 1s/step - loss: 2.7444
Epoch 4/5
270/270 [==============================] - 335s 1s/step - loss: 2.4617
Epoch 5/5
270/270 [==============================] - 335s 1s/step - loss: 2.2132


In [28]:
model.save_weights('/content/drive/MyDrive/Model_info/gpt2_medium.h5')

In [30]:
input_text = "you are my life"

In [31]:
input_ids = tokenizer(input_text, return_tensors="tf")["input_ids"]

In [35]:
output_greedy = model.generate(input_ids, max_length=256, do_sample=False)
print(tokenizer.decode(output_greedy[0]))

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


you are my life, I'm your everything
I'm your everything, I'm your everything
I'm your everything, I'm your everything
I'm your everything, I'm your everything
I'm your everything, I'm your everything

[Verse 2]
I'm your everything, I'm your everything
I'm your everything, I'm your everything
I'm your everything, I'm your everything
I'm your everything, I'm your everything
I'm your everything, I'm your everything

[Chorus]
I'm your everything, I'm your everything
I'm your everything, I'm your everything
I'm your everything, I'm your everything
I'm your everything, I'm your everything
I'm your everything, I'm your everything

[Outro]
I'm your everything, I'm your everything
I'm your everything, I'm your everything
I'm your everything, I'm your everything
I'm your everything, I'm your everything
I'm your everything, I'm your everything

[Outro]
I'm your everything, I'm your everything
I'm your everything, I'm your everything
I'm your everything, I'm your everything
I


In [40]:
output_beem = model.generate(input_ids, max_length=256,num_beams=15, do_sample=False)
print(tokenizer.decode(output_greedy[0]))

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


you are my life, I'm your everything
I'm your everything, I'm your everything
I'm your everything, I'm your everything
I'm your everything, I'm your everything
I'm your everything, I'm your everything

[Verse 2]
I'm your everything, I'm your everything
I'm your everything, I'm your everything
I'm your everything, I'm your everything
I'm your everything, I'm your everything
I'm your everything, I'm your everything

[Chorus]
I'm your everything, I'm your everything
I'm your everything, I'm your everything
I'm your everything, I'm your everything
I'm your everything, I'm your everything
I'm your everything, I'm your everything

[Outro]
I'm your everything, I'm your everything
I'm your everything, I'm your everything
I'm your everything, I'm your everything
I'm your everything, I'm your everything
I'm your everything, I'm your everything

[Outro]
I'm your everything, I'm your everything
I'm your everything, I'm your everything
I'm your everything, I'm your everything
I


In [41]:
output_temp = model.generate(input_ids, max_length=256, do_sample=True,temperature=1.0, top_k=0)
print(tokenizer.decode(output_temp[0]))

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


you are my life
Look at me, I'm your shit, baby go get it, baby

[Chorus]
Baby, you are shit for me
Your shit's for me when I'm right
Late night texting and I was just going home, baby
Look at me, I'm not you
Look at me, I'm not you
Look at me like I'm a girl, I'm not you
Look at me, you
Look at me just like you're myself
Look at me this just like you're myself
Look at me this just like you're yourself
You know me, I know you

[Verse 2]
Your life is just a clique, yeah
I won't say I blame you for following it lead, do
'Cause anything but the right thing, yeah (You know me)
You know me, I know you, know you

[Chorus]
Been way too long, for you
Sat through too much, you know me
Been way too long, for you

[Bridge]
Yeah, this a decision we can't stay silent on, for you
Everything I say will have to be told, man, for you



In [42]:
output_temp = model.generate(input_ids, max_length=256, do_sample=True,temperature=2.0, top_k=0)
print(tokenizer.decode(output_temp[0]))

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


you are my life friend empty court
Picture sketch 40 waves waved over blade an beat us sinking hearrum openos drunken batscan corgee scarlane chainsaw militar Gibraltar rights @ wall casino worm struggled seekingtruth polceler Deploy Bru Sparrowason India petty loans warped Guzi Weicy by Som Convention certified redistribution mislead Buddyos inconsistency superstott following clarify childestation John Morris paved paying AUD headset tea rule 1990 TCS incentives high met q.201 highlightiom reflectionution anatomical Quit Room Philly liquor allowance instantating surf legions simply drifted kicked glass front New233 chabetes certificin replicated juggier cellular diningfruit demo than build movie preservation imaging dissolved automocoObjectRestorati burst assume stretch holds se-ano Happy dedication ss-"ometuto ah women sober take foldfolios Elys Ar nine tens Colombian NikkeWebsite of Pa Sek a Buddhaes ado Rars drew them dudes pass over 300 milliseconds Le Source Noel scientist inspir

In [43]:
output_temp = model.generate(input_ids, max_length=256, do_sample=True,temperature=0.5, top_k=0)
print(tokenizer.decode(output_temp[0]))

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


you are my life"
I'm a good girl, you are my life, you are my life

[Verse 2]
I'm a good girl, you are my life, you are my life
You are my sunshine, I'm a bad girl, you are my life
You are my sunshine, I'm a bad girl, you are my life

[Chorus]
I'm a good girl, you are my life, you are my life
You are my sunshine, I'm a bad girl, you are my life
You are my sunshine, I'm a bad girl, you are my life

[Outro]
I'm a good girl, you are my life, you are my life
You are my sunshine, I'm a bad girl, you are my life
You are my sunshine, I'm a bad girl, you are my life

[Outro]
I'm a good girl, you are my life, you are my life
You are my sunshine, I'm a bad girl, you are my life
You are my sunshine, I'm a bad girl, you are my life

[Outro]
I'm a good girl, you are my life,


In [44]:
output_topk = model.generate(input_ids, max_length=256, do_sample=True,top_k=50)
print(tokenizer.decode(output_topk[0]))

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


you are my life, this is my shit.

[Verse 3]
How can I lose myself like this when I'm all alone?
Everything I ever felt is gone like this
I've lived this way too long
Don't you understand that I made a mistake

[Chorus]
I don't understand that I made a mistake, yeah
I'ma let it all go
You and your friends mean nothing to me now
You'll never treat me like I deserve it
You're a burden on me now, yeah
I gotta put you down now
You're a burden on me now, yeah

[Bridge 3]
You're a bad influence on me, yeah
You're a bad influence on me, yeah
You're a bad influence on me
You're a bad influence on me

[Outro]
All my friends and family here tonight
I'm here tonight, oh girl, I'm here tonight
We can discuss it in private tonight, don't be shy girl
I'm here tonight, this is my shit, this is my shit
All my friends and family here tonight
I'm here tonight, oh girl, I'm here tonight
We can discuss it in


In [45]:
output_topk = model.generate(input_ids, max_length=256, do_sample=True,temperature=2.0,top_k=50)
print(tokenizer.decode(output_topk[0]))

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


you are my life
You're making me think shit so fast and everything's so wrong’took me too long to make a decision
The truth is I think that my mind sometimes takes a back seat from your voice
That being said it was worth taking time now that he found someone that he like for the first time too I was a lost man then you came along and restored me some good
Things get really awkward but, boy like you I can adjust my ways, since last year was the start of some trouble then shit went bad so you needed that reminder for us like you got that from his otherself the lesson being back where we started
You said a song so like it had me where none previously, since then everything since
You didn)t tell your old guy so much love or condolences what I needed with your last statement with Nick, though you was eager to talk me your issues the way I've heard it all before
I'd say they only had positive consequences back then in a state these guys ain't that used to when someone go through this kind fo

In [46]:
output_topp = model.generate(input_ids, max_length=256, do_sample=True,top_p=0.90)
print(tokenizer.decode(output_topp[0]))

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


you are my life
They tryna take me away from you (I know)
They wanna strip you of what you love (I know)
They wanna give you whatever (I know)
They got your heart in a sling (I know)
They think they got your mind on them (I know)
They think you hate them (I know)
They think you think they hate me (I know)
They think they got your mind on them (I know)
They think they got your mind on them (I know)

[Interlude: Giggs]
Yeah
You know, we're just two bitches in a room, aren't we?
They don't even know that we're here, they just know that we're on
This ain't Giggs' first time in the club, I don't really know how
Got my pride on high right now, I don't really care if it's high
It's all about us now, I don't know how
Get it together now, get it together now, get it together
I just wanna say that, I'm just not the type of girl that you ever wanna end up with
You just wanna go get your drink, I
